In [1]:
#!pip install tqdm
import torchvision.models as models
import sys
import torch
from torch import nn
import time
from tqdm import tqdm
import torch.optim as optim
import copy

# Specify where to find the data preparation class
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [2]:
# VGG16 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
DIMENSIONS = 3
#SIZE = 256

LR = 0.1
MOMENTUM=0.85
WEIGHT_DECAY = 1e-4
LR_STEP_SIZE = 5
LR_GAMMA = 0.1

In [3]:
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="train", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader
test_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="test", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(train_data_loader):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([3, 3, 3, 5, 5])
Batch Index: 1
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([0, 5, 6, 1, 6])
Batch Index: 2
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([3, 3, 2, 5, 3])


In [4]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
vgg19 = models.vgg19(pretrained=True)


C:\Users\mk6812ct\Downloads\Facial_Emotion_Detec-main (1)\Facial_Emotion_Detec-main\.env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mk6812ct\Downloads\Facial_Emotion_Detec-main (1)\Facial_Emotion_Detec-main\.env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
for param in vgg19.parameters():
    param.requires_grad = False
dataloaders = {'train':train_data_loader, 'val':test_data_loader}


In [6]:
num_classes = len(train_data_loader.dataset.classes)
vgg19.classifier[6] = nn.Linear(4096, num_classes)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    vgg19.parameters(),
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)

LR_SCHEDULER = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP_SIZE, gamma=LR_GAMMA)

optimizer_vgg19 = optim.Adam(vgg19.parameters())
num_epochs = 25
for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}'.format(epoch,num_epochs -1))
    print('-'*10)
    total_batch = len(train_data_loader.dataset) // 16
    for phase in ['train', 'val']:
        if phase == 'train':
            vgg19.train()
        else:
            vgg19.eval()

        running_loss = 0.0
        corrects = 0

        for i,(inputs, labels) in enumerate(dataloaders[phase]):
            optimizer_vgg19.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs_vgg19 = vgg19(inputs)
                _, preds_vgg19 = torch.max(outputs_vgg19, 1)

                loss_vgg19 = criterion(outputs_vgg19, labels)

                if phase == 'train':
                    loss_vgg19.backward()
                    optimizer_vgg19.step()

                    if (i+1) % 5 == 0:
                        print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                             %(epoch+1, num_epochs, i+1, total_batch, loss_vgg19.item()))

            running_loss += loss_vgg19.item() * inputs.size(0)
            corrects += torch.sum(preds_vgg19 == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase])
        epoch_acc = corrects.double() / len(dataloaders[phase])

        LR_SCHEDULER.step()

        print('{} Loss: {:.4f} | Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 0/24
----------
Epoch [1/25], lter [5/1752] Loss: 1.7369
Epoch [1/25], lter [10/1752] Loss: 2.0329
Epoch [1/25], lter [15/1752] Loss: 1.7283
Epoch [1/25], lter [20/1752] Loss: 2.1072
Epoch [1/25], lter [25/1752] Loss: 1.7550
Epoch [1/25], lter [30/1752] Loss: 2.5060
Epoch [1/25], lter [35/1752] Loss: 1.9869
Epoch [1/25], lter [40/1752] Loss: 1.9585
Epoch [1/25], lter [45/1752] Loss: 1.8896
Epoch [1/25], lter [50/1752] Loss: 1.5373
Epoch [1/25], lter [55/1752] Loss: 1.5398
Epoch [1/25], lter [60/1752] Loss: 1.8449
Epoch [1/25], lter [65/1752] Loss: 1.5749
Epoch [1/25], lter [70/1752] Loss: 1.8318
Epoch [1/25], lter [75/1752] Loss: 1.8356
Epoch [1/25], lter [80/1752] Loss: 1.6900
Epoch [1/25], lter [85/1752] Loss: 2.0938
Epoch [1/25], lter [90/1752] Loss: 1.6288
Epoch [1/25], lter [95/1752] Loss: 1.6841
Epoch [1/25], lter [100/1752] Loss: 1.6234
Epoch [1/25], lter [105/1752] Loss: 2.2852
Epoch [1/25], lter [110/1752] Loss: 1.6101
Epoch [1/25], lter [115/1752] Loss: 1.8793
Epoch [1/

C:\Users\mk6812ct\Downloads\Facial_Emotion_Detec-main (1)\Facial_Emotion_Detec-main\.env\Lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


train Loss: 28.0343 | Acc: 5.3360


  4%|▍         | 1/25 [58:06<23:14:47, 3486.96s/it]

...

val Loss: 25.5151 | Acc: 6.1581
Epoch 23/24
----------
Epoch [24/25], lter [5/1752] Loss: 1.8486
Epoch [24/25], lter [10/1752] Loss: 1.6745
Epoch [24/25], lter [15/1752] Loss: 2.0794
Epoch [24/25], lter [20/1752] Loss: 1.6564
Epoch [24/25], lter [25/1752] Loss: 1.6421
Epoch [24/25], lter [30/1752] Loss: 1.7735
Epoch [24/25], lter [35/1752] Loss: 2.1567
Epoch [24/25], lter [40/1752] Loss: 1.9395
Epoch [24/25], lter [45/1752] Loss: 1.9071
Epoch [24/25], lter [50/1752] Loss: 1.6626
Epoch [24/25], lter [55/1752] Loss: 1.5580
Epoch [24/25], lter [60/1752] Loss: 1.5304
Epoch [24/25], lter [65/1752] Loss: 1.6595
Epoch [24/25], lter [70/1752] Loss: 1.8228
Epoch [24/25], lter [75/1752] Loss: 1.4163
Epoch [24/25], lter [80/1752] Loss: 2.0145
Epoch [24/25], lter [85/1752] Loss: 1.7352
Epoch [24/25], lter [90/1752] Loss: 1.4656
Epoch [24/25], lter [95/1752] Loss: 1.8034
Epoch [24/25], lter [100/1752] Loss: 1.7015
Epoch [24/25], lter [105/1752] Loss: 1.5265
Epoch [24/25], lter [110/1752] Loss: 1.63

 96%|█████████▌| 24/25 [18:36:23<46:04, 2764.80s/it]  

val Loss: 24.8446 | Acc: 6.6615
Epoch 24/24
----------
Epoch [25/25], lter [5/1752] Loss: 1.9637
Epoch [25/25], lter [10/1752] Loss: 1.9563
Epoch [25/25], lter [15/1752] Loss: 1.7026
Epoch [25/25], lter [20/1752] Loss: 1.5719
Epoch [25/25], lter [25/1752] Loss: 1.6751
Epoch [25/25], lter [30/1752] Loss: 1.5102
Epoch [25/25], lter [35/1752] Loss: 1.8928
Epoch [25/25], lter [40/1752] Loss: 1.6557
Epoch [25/25], lter [45/1752] Loss: 1.7092
Epoch [25/25], lter [50/1752] Loss: 1.5225
Epoch [25/25], lter [55/1752] Loss: 2.4776
Epoch [25/25], lter [60/1752] Loss: 2.2645
Epoch [25/25], lter [65/1752] Loss: 1.8877
Epoch [25/25], lter [70/1752] Loss: 2.2053
Epoch [25/25], lter [75/1752] Loss: 1.4510
Epoch [25/25], lter [80/1752] Loss: 2.1562
Epoch [25/25], lter [85/1752] Loss: 1.7868
Epoch [25/25], lter [90/1752] Loss: 1.6756
Epoch [25/25], lter [95/1752] Loss: 1.8254
Epoch [25/25], lter [100/1752] Loss: 2.2782
Epoch [25/25], lter [105/1752] Loss: 1.2339
Epoch [25/25], lter [110/1752] Loss: 1.89

100%|██████████| 25/25 [19:22:25<00:00, 2789.80s/it]

val Loss: 24.6747 | Acc: 6.4922


In [7]:
# Saving the model
torch.save(vgg19.state_dict(), 'trained_vgg19.pt')

In [8]:
# Initialize the correct_predictions and total_predictions variables
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['val']:
    # Forward pass
    inputs, target = batch_data
    output = vgg19(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 41.19%
